In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
import graphviz
from graphviz import Source
from sklearn import tree
from IPython.display import Image
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier

import os

from IPython.core.display import display, HTML, Javascript
from string import Template
import json
import IPython.display

df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

df_train = df_train.head(1459)
df_train.head()

In [ ]:
print(df_train.columns)
print(df_train.shape, df_test.shape)

In [ ]:
df_train.describe()

In [ ]:
df_train[['LotArea','YearBuilt','SalePrice']].describe()

In [ ]:
y = df_train['SalePrice']
X = df_train[['LotArea','1stFlrSF','2ndFlrSF','OverallQual','OverallCond','YearBuilt','GrLivArea','FullBath','HalfBath','TotRmsAbvGrd','PoolArea']]
test_X = df_test[['LotArea','1stFlrSF','2ndFlrSF','OverallQual','OverallCond','YearBuilt','GrLivArea','FullBath','HalfBath','TotRmsAbvGrd','PoolArea']]

In [ ]:
hp_model = DecisionTreeRegressor()
hp_model.fit(X, y)

In [ ]:
print("Making predictions for the following 5 houses:")
print(test_X.head())
print("The predictions are")
print(hp_model.predict(test_X.head()))

In [ ]:
graph = Source(tree.export_graphviz(hp_model, out_file=None, feature_names=test_X.columns,max_depth=3,impurity=False,rounded=True,precision=1))
png_bytes = graph.pipe(format='png')
with open('dtree_pipe.png','wb') as f:
    f.write(png_bytes)
Image(png_bytes)

In [ ]:
predicted_home_prices = hp_model.predict(test_X)
mean_absolute_error(y, predicted_home_prices)

In [ ]:
def get_mae(max_leaf_nodes, predictors_train, targ_train):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(predictors_train, targ_train)
    preds_val = model.predict(predictors_train)
    mae = mean_absolute_error(targ_train, preds_val)
    return(mae)

# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [1200, 1300, 1400, 1500, 1700]:
    my_mae = get_mae(max_leaf_nodes, test_X, y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

In [ ]:
forest_model = RandomForestRegressor()
forest_model.fit(test_X, y)
model_preds = forest_model.predict(test_X)
print(mean_absolute_error(y, model_preds))

In [ ]:
model_preds = hp_model.predict(test_X)
print(mean_absolute_error(y, model_preds))

In [ ]:
xgbmodel = XGBClassifier()
xgbmodel.fit(test_X, y)
model_preds = xgbmodel.predict(test_X)
print(mean_absolute_error(y, model_preds))

In [ ]:
predicted_prices = hp_model.predict(test_X)
my_submission = pd.DataFrame({'Id': df_test.Id, 'SalePrice': predicted_prices})
my_submission.to_csv('submission.csv', index=False)

In [ ]:
my_submission.head()